# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion. 

## Entry data : CSV file containing the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId


#### Creating a Cluster

In [76]:
# make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [77]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [78]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### 1. Query 1 : artist, song title and song's length in the music app history that was heard during  sessionId = ?, and itemInSession  = ?

* Expected output : artist, song, length
* Based on : sessionId, itemInSession => primary key

In [96]:
query = "CREATE TABLE IF NOT EXISTS song_where_sessionId_and_itemInSession "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, \
PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [97]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_where_sessionId_and_itemInSession (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Define a utility function

In [120]:
def get_song_on_sessionId_and_itemInSession(sessionId, itemInSession) :
    query1 = f"select artist, song, length from song_where_sessionId_and_itemInSession \
    WHERE sessionId = {sessionId} and itemInSession = {itemInSession};"
    
    try:
        rows = session.execute(query1)
    except Exception as e:
        print(e)

    for row in rows:
        print (f"artist : {row.artist} / song : {row.song} / lentgth : {row.length}")

In [121]:
sessionId = 338
itemInSession = 4
get_song_on_sessionId_and_itemInSession(sessionId, itemInSession) 

artist : Faithless / song : Music Matters (Mark Knight Dub) / lentgth : 495.30731201171875


### 2. Query 2 : name of artist, song (sorted by itemInSession) and user (first and last name) for userid = ?, sessionid = ?

* Expected output : artist, song, user
* Based on : userid, sessionid => primary key = ((userid, sessionid), itemInSession)

The tuple (userid, sessionid) is not sufficient to create a primary key because a user may listen to several songs during a session. This is why we will add itemInSession to make sure every row is unique. What's more, sessions belonging to the same user might be in different nodes with a potential performance issue if the database is very large. Therefore we will use both userId and sessionId as partition keys so sessions from the same user are stored together.

In [124]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
details = "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
PRIMARY KEY ((userId, sessionId), itemInSession)) ;"
query = query + details
try:
    session.execute(query)
except Exception as e:
    print(e)

In [132]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Define a utility function

In [135]:
def get_song_playlist_session(userId, sessionId) :
    query2 = f"SELECT artist, song, firstName, lastName FROM song_playlist_session \
    WHERE userId = {userId} and sessionId = {sessionId} \
    ORDER BY itemInSession ;" \
    
    try:
        rows = session.execute(query2)
    except Exception as e:
        print(e)

    for row in rows:
        print (f"artist : {row.artist} / song : {row.song} / user : {row.firstname} {row.lastname}")

In [136]:
userId = 10
sessionId = 182
get_song_playlist_session(userId, sessionId) 

artist : Down To The Bone / song : Keep On Keepin' On / user : Sylvie Cruz
artist : Three Drives / song : Greece 2000 / user : Sylvie Cruz
artist : Sebastien Tellier / song : Kilometer / user : Sylvie Cruz
artist : Lonnie Gordon / song : Catch You Baby (Steve Pitron & Max Sanna Radio Edit) / user : Sylvie Cruz


### 3. Query 3 : every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

* Expected output : first and last name
* Based on : song => primary key = (song, sessionId, itemInSession)

song alone is definitely not a good primary key. The tuple (song, sessionid) is not sufficient either to create a primary key because a user may listen to the same song several times during a same session. This is why we will add itemInSession to make sure every row is unique.

In [89]:
query = "CREATE TABLE IF NOT EXISTS user_where_song "
details = "(song text, sessionId int, itemInSession int, firstName text, lastName text, \
PRIMARY KEY (song, sessionId, itemInSession))"
query = query + details
try:
    session.execute(query)
except Exception as e:
    print(e)

In [91]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_where_song (song, sessionId, itemInSession, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[8]), int(line[3]), line[1], line[4]))

In [92]:
def get_user_where_song(song_name) :
    query3 = f"select firstName, lastName from user_where_song WHERE song = '{song_name}';" 
    try:
        rows = session.execute(query3)
    except Exception as e:
        print(e)

    for row in rows:
        print (row.firstname, row.lastname)

In [93]:
song_name = 'All Hands Against His Own'
get_user_where_song(song_name) 

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Drop the tables before closing out the sessions

In [137]:
query = "drop table song_where_sessionId_and_itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
query = "drop table user_where_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [138]:
session.shutdown()
cluster.shutdown()